In [1]:
import os
import torch
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
os.environ['KMP_DUPLICATE_LIB_OK']="True"
%matplotlib inline
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def block(input_dim,out_dim):
    return nn.Sequential(
    nn.Linear(input_dim,out_dim),
    nn.BatchNorm1d(out_dim),
    nn.ReLU()
    )

In [3]:
class Model(nn.Module):
    def __init__(self,input_dim,num_classes,hidden_dim):
        super(Model,self).__init__()
        self.net=nn.Sequential(
        block(input_dim,hidden_dim*2),
        block(hidden_dim*2,num_classes)
        )
    def forward(self,x):
        return self.net(x)

In [4]:
x=torch.randn(50,784)
model=Model(784,10,128)
print(model(x).shape)

torch.Size([50, 10])


In [5]:
dataset=datasets.MNIST('.',train=True,download=False,transform=transforms.ToTensor())
data=DataLoader(dataset,batch_size=128,shuffle=True)

In [6]:
n_epoch=1
input_size=784
num_classes=10
hidden_units=128
criterion=nn.CrossEntropyLoss()
model=Model(input_size,num_classes,hidden_units).to(device)
optimizer=optim.Adam(model.parameters(),lr=.001)

In [7]:
for epoch in range(n_epoch):
    for images,labels in tqdm(data):
        image=images.reshape(images.shape[0],-1)
        images=images.to(device)
        labels=labels.to(device)
        pred=model(image)
        loss=criterion(pred,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [8]:
def acc(data,model):
    num_correct=0
    num_sample=0
    model.eval()
    with torch.no_grad():
        for image,label in data:
            image=image.to(device)
            label=label.to(device)
            image=image.reshape(image.shape[0],-1)
            _,pred=model(image).max(1)
            num_correct+=(pred==label).sum()
            num_sample+=pred.size(0)
        acc=num_correct/num_sample
        print(acc)

In [9]:
acc(data,model)

tensor(0.9739)


In [10]:
dataset=datasets.MNIST('.',train=False,download=False,transform=transforms.ToTensor())
test_data=DataLoader(dataset,batch_size=128,shuffle=True)

In [11]:
acc(test_data,model)

tensor(0.9679)
